# Project 2 Team 3 Technical Document 

The aim  this project was to develop a data resource that enables analysis of COVID cases by postcode and Local Government Area (LGA) compared to the population and population density of each LGA.

Cleaned df with help of - line number XX cahnge / add 

Data Sources:

Three data sets were sourced to undertake this analysis

1. Commonwealth Australian Beureau of Statisitcs (ABS), population by LGA
2. LGA Mapping file 
3. The Victorian DHHS COVID data:

1. Commonwealth Australian Beureau of Statisitcs (ABS), population by LGA
        (a) data soure - url:"https://api.data.abs.gov.au/data/C21_G04_LGA/....2"
        (b) Format: Data set sourced via API
        (c) Extract and Load method: Sourced via abs API 
        (d) Original data fields: DATAFLOW,	AGEINGP: Age,	SEXP: Sex, 	REGION: Region,	REGION_TYPE: Region Type	        STATE: State,	TIME_PERIOD: Time Period,	OBS_VALUE
        (e) Data transformation: 
              (i) REGION: Region:  separate data in REGION: Region column into 'Regions ID' and 'Region Name'
              (ii) Drop DATAFLOW, AGEINGP: Age, REGION: Region, STATE: State
              (iii) SEXP: Sex filter and retain 3: persons value in SEXP: Sex column - this removed the male and                       female categores and enabled to calculation of the total population
              (v)Rename 'OBS_VALUE' to 'Population'
              (vi) Drop 'persons' column to avoid duplicating data         
        (f) Comments
                This data set could not be directly merged with the Victorian DHHS COVID data set. The                             'Region Name' field in this data set did not include suffic (x) that was present in the                           'Localgovernmentarea' field. 
        (g) Final data fields: Population	Region ID (PRIMARY KEY)	Region Name
 
        
 2. LGA Mapping and LGA SQKM file 
         (a) data source: Resource/'LGA_2020_VIC2.csv'
         (b) Format: CSV 
         (c) Extract and load method: data downloaded from Vic Government websit 
         (d) Original Data Fields: MB_CODE_2016	LGA_CODE_2020	LGA_NAME_2020	STATE_CODE_2016	STATE_NAME_2016	                AREA_ALBERS_SQKM
         (e) Data transformation
             (i) Drop MB_CODE_2016 STATE_CODE_2016	STATE_NAME_2016	
         (f) Comments 
             This data set was required to map the LGAs in the abs and COVID data sets and provide not be directly              This data set also provides the area square kilometer data to enable population density for each LGA              to be shown
         (g) Final data fields: LGA_CODE_2020 int (PRIMARY KEY)	LGA_NAME_20201 object (FORIEGN KEY)	                            AREA_ALBERS_SQKM float

1. The Victorian DHHS COVID that shows the number of COVID cases by LGA by date (day, month and year) 
        (a) data source: "https://www.dhhs.vic.gov.au/ncov-covid-cases-by-lga-source-csv"
        (b) Format: CSV 
        (c) Extract and Load method: read in directly from the DHHS web site as too large to download and save
        (d) Original data fields: diagnosis_date,Postcode,	acquired,	Localgovernmentarea.
        (e) Data transformation: 
              (i) use the 'diagnosis_date' to create a column called 'year' to enable the volume of people to be                       grouped by year 
              (ii) drop the 'acquired' field
        (f) Comments: 
                This data set could not be directly merged with the abs population data set. The                                   'Localgovernmentarea' field in this data set did not include the LGA numerical ID and contained an                  additional code in the form of(x)at the end of the name. This code was not present in the abs                      data set to enable data to be merged a third data set is required to match the LGA name and ID .                  This data set did not show population density 
         (g) Final Data Fields 

This resource forms the foundations which, at a future date, can enable the addition and analysis of other socio, demographic economic factors that are captured in Census data and other State and Commonwealth data sets

SQL

ERD Diagram 


In [21]:
#import dependencies 
import json
import requests
import pandas as pd
from io import StringIO
import numpy as np
import warnings
warnings.filterwarnings('ignore')


In [22]:
# Review abs API documentation adn liase with abs data custodians to identify required data points to source
# population data data from abs API 
url = "https://api.data.abs.gov.au/data/C21_G04_LGA/....2"

In [23]:
#Source Population data from ABS API and review data fiel and data in top 5 rows
headers = {"Accept": "application/vnd.sdmx.data+csv;labels=both"}
req = requests.get(url, headers=headers)
data = StringIO(req.text)

abs_df = pd.read_csv(data)
abs_df.head()

,DATAFLOW,AGEINGP: Age,SEXP: Sex,REGION: Region,REGION_TYPE: Region Type,STATE: State,TIME_PERIOD: Time Period,OBS_VALUE
0,ABS:C21_G04_LGA(1.0.0),24: 24,3: Persons,24210: Manningham,LGA2021: Local Government Areas (2021),2: Victoria,2021,1459
1,ABS:C21_G04_LGA(1.0.0),58: 58,2: Females,24410: Maroondah,LGA2021: Local Government Areas (2021),2: Victoria,2021,764
2,ABS:C21_G04_LGA(1.0.0),60: 60,1: Males,25250: Moreland,LGA2021: Local Government Areas (2021),2: Victoria,2021,759
3,ABS:C21_G04_LGA(1.0.0),32: 32,1: Males,25430: Mount Alexander,LGA2021: Local Government Areas (2021),2: Victoria,2021,89
4,ABS:C21_G04_LGA(1.0.0),60: 60,3: Persons,25710: Nillumbik,LGA2021: Local Government Areas (2021),2: Victoria,2021,970


In [24]:
#split REGION: Region field to create 'Region ID' field 
abs_df["Region ID"]=abs_df["REGION: Region"].map(lambda x:x.split(":")[0])
abs_df.head()

,DATAFLOW,AGEINGP: Age,SEXP: Sex,REGION: Region,REGION_TYPE: Region Type,STATE: State,TIME_PERIOD: Time Period,OBS_VALUE,Region ID
0,ABS:C21_G04_LGA(1.0.0),24: 24,3: Persons,24210: Manningham,LGA2021: Local Government Areas (2021),2: Victoria,2021,1459,24210
1,ABS:C21_G04_LGA(1.0.0),58: 58,2: Females,24410: Maroondah,LGA2021: Local Government Areas (2021),2: Victoria,2021,764,24410
2,ABS:C21_G04_LGA(1.0.0),60: 60,1: Males,25250: Moreland,LGA2021: Local Government Areas (2021),2: Victoria,2021,759,25250
3,ABS:C21_G04_LGA(1.0.0),32: 32,1: Males,25430: Mount Alexander,LGA2021: Local Government Areas (2021),2: Victoria,2021,89,25430
4,ABS:C21_G04_LGA(1.0.0),60: 60,3: Persons,25710: Nillumbik,LGA2021: Local Government Areas (2021),2: Victoria,2021,970,25710


In [25]:
#split REGION: Region field to create 'Region Name' field 
abs_df["Region Name"]=abs_df["REGION: Region"].map(lambda x:x.split(":")[1])
abs_df.head()

,DATAFLOW,AGEINGP: Age,SEXP: Sex,REGION: Region,REGION_TYPE: Region Type,STATE: State,TIME_PERIOD: Time Period,OBS_VALUE,Region ID,Region Name
0,ABS:C21_G04_LGA(1.0.0),24: 24,3: Persons,24210: Manningham,LGA2021: Local Government Areas (2021),2: Victoria,2021,1459,24210,Manningham
1,ABS:C21_G04_LGA(1.0.0),58: 58,2: Females,24410: Maroondah,LGA2021: Local Government Areas (2021),2: Victoria,2021,764,24410,Maroondah
2,ABS:C21_G04_LGA(1.0.0),60: 60,1: Males,25250: Moreland,LGA2021: Local Government Areas (2021),2: Victoria,2021,759,25250,Moreland
3,ABS:C21_G04_LGA(1.0.0),32: 32,1: Males,25430: Mount Alexander,LGA2021: Local Government Areas (2021),2: Victoria,2021,89,25430,Mount Alexander
4,ABS:C21_G04_LGA(1.0.0),60: 60,3: Persons,25710: Nillumbik,LGA2021: Local Government Areas (2021),2: Victoria,2021,970,25710,Nillumbik


In [26]:
#df[["3: Persons","2: Females","1:Males"]] = df['SEXP: Sex'].str.extract(r'\["(?:([^"]*)",")?(\d+)(?:","(\d+))?"',expand=False)
#df.head()

In [27]:
#drop round of unwanted items from abs_df

abs_df.drop(columns=["DATAFLOW","AGEINGP: Age","REGION_TYPE: Region Type","STATE: State","TIME_PERIOD: Time Period"], inplace= True)
abs_df.head()



,SEXP: Sex,REGION: Region,OBS_VALUE,Region ID,Region Name
0,3: Persons,24210: Manningham,1459,24210,Manningham
1,2: Females,24410: Maroondah,764,24410,Maroondah
2,1: Males,25250: Moreland,759,25250,Moreland
3,1: Males,25430: Mount Alexander,89,25430,Mount Alexander
4,3: Persons,25710: Nillumbik,970,25710,Nillumbik


In [28]:
#Sex filter and retain 3: persons value in SEXP: Sex column - 
#this removed the male and female categores and enabled to calculation of the total population
abs_clean_df=abs_df[abs_df["SEXP: Sex"] == '3: Persons'] 
abs_clean_df.head()

,SEXP: Sex,REGION: Region,OBS_VALUE,Region ID,Region Name
0,3: Persons,24210: Manningham,1459,24210,Manningham
4,3: Persons,25710: Nillumbik,970,25710,Nillumbik
5,3: Persons,25900: Port Phillip,2467,25900,Port Phillip
6,3: Persons,26260: Southern Grampians,852,26260,Southern Grampians
9,3: Persons,29399: Unincorporated Vic,7,29399,Unincorporated Vic


In [29]:
abs_clean_df.describe()

,OBS_VALUE
count,8364.000000
mean,2298.721186
std,11206.035369
min,0.000000
25%,207.000000
50%,681.500000
75%,1794.000000
max,365239.000000


In [30]:
abs_clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8364 entries, 0 to 25089
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   SEXP: Sex       8364 non-null   object
 1   REGION: Region  8364 non-null   object
 2   OBS_VALUE       8364 non-null   int64 
 3   Region ID       8364 non-null   object
 4   Region Name     8364 non-null   object
dtypes: int64(1), object(4)
memory usage: 392.1+ KB


In [31]:
#Rename 'OBS_VALUE' to 'Population' dfmi.loc[:, ('one', 'second')]

abs_clean_df.rename(columns={'OBS_VALUE': 'Population'}, inplace=True)
abs_clean_df.head()


,SEXP: Sex,REGION: Region,Population,Region ID,Region Name
0,3: Persons,24210: Manningham,1459,24210,Manningham
4,3: Persons,25710: Nillumbik,970,25710,Nillumbik
5,3: Persons,25900: Port Phillip,2467,25900,Port Phillip
6,3: Persons,26260: Southern Grampians,852,26260,Southern Grampians
9,3: Persons,29399: Unincorporated Vic,7,29399,Unincorporated Vic


In [32]:
#Drop 'persons' column to avoid duplicating data
abs_clean_df.drop(columns=["SEXP: Sex","REGION: Region"], inplace= True)
abs_clean_df.head()



,Population,Region ID,Region Name
0,1459,24210,Manningham
4,970,25710,Nillumbik
5,2467,25900,Port Phillip
6,852,26260,Southern Grampians
9,7,29399,Unincorporated Vic


In [33]:
abs_clean_df.describe()

,Population
count,8364.000000
mean,2298.721186
std,11206.035369
min,0.000000
25%,207.000000
50%,681.500000
75%,1794.000000
max,365239.000000


In [34]:
abs_clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8364 entries, 0 to 25089
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Population   8364 non-null   int64 
 1   Region ID    8364 non-null   object
 2   Region Name  8364 non-null   object
dtypes: int64(1), object(2)
memory usage: 261.4+ KB


In [35]:
#Convert abs_clean data set to a csv format
abs_clean_df.to_csv("abs_clean.csv")

In [36]:
# need to chaeck unnamed field 
abs_clean=pd.read_csv("abs_clean.csv")
abs_clean.head()

,Unnamed: 0,Population,Region ID,Region Name
0,0,1459,24210,Manningham
1,4,970,25710,Nillumbik
2,5,2467,25900,Port Phillip
3,6,852,26260,Southern Grampians
4,9,7,29399,Unincorporated Vic


In [37]:
#???
abs_clean_df = abs_clean_df.apply(lambda x: "{:},".format(x))


In [38]:
abs_clean_df.head()

Population     0        1459\n4         970\n5        2467\n6...
Region ID      0        24210\n4        25710\n5        25900...
Region Name    0                 Manningham\n4               ...
dtype: object

In [39]:
#?? DELETE API - Population
#response = requests.get(url)
#print(response.url)

In [40]:
#Put ?? DELETE ABS data into DataFrame 

#abs_data = pd.read_csv('https://api.data.abs.gov.au/data/ABS,C21_G04_LGA,1.0.0/....2?dimensionAtObservation=AllDimensions&format=csv')
#print(abs_data)
#abs_data.head()


In [41]:
#?? DELETE API
#abs_data.loc[abs_data["STATE"]==2]

In [42]:
#?? DELETE ABS clean data - API - Total population 

#abs_data_clean= pd.DataFrame(abs_data)

#print(abs_data_clean.groupby(["OBS_VALUE"]).sum())


In [43]:
#?? DELETE api
#abs_data['STATE'].sum()

In [44]:
##?? DELETE CSV0 - Adding new CSV - LGA - Population

In [45]:
#?? DELETE ANS data ? delete CSV0
#df = pd.read_csv('LGAVICPOP/2021Census_G01_VIC_LGA.csv')
#df.head()

In [46]:
#?? DELETE clean data - dropping unwanted items from dataframe

#population_LGA = df[['LGA_CODE_2021','Tot_P_P']].copy()
#print(population_LGA)


In [47]:
#?? DELETE Providing the total sum of population 
#population_LGA['Tot_P_P'].sum()


In [48]:
# Read in  LGA Mapping and LGA SQKM file
lga_mapping_df = pd.read_csv('LGA_2020_VIC2.csv')
lga_mapping_df.head()

,MB_CODE_2016,LGA_CODE_2020,LGA_NAME_2020,STATE_CODE_2016,STATE_NAME_2016,AREA_ALBERS_SQKM
0,20000009499,29499,No usual address (Vic.),2,Victoria,0.0000
1,20000010000,20110,Alpine (S),2,Victoria,0.0127
2,20000021000,20110,Alpine (S),2,Victoria,0.0127
3,20000022000,20110,Alpine (S),2,Victoria,0.0105
4,20000023000,20110,Alpine (S),2,Victoria,0.0105


In [49]:
#drop unwnated items from df1
lga_mapping_df.drop(columns=["MB_CODE_2016","STATE_CODE_2016","STATE_NAME_2016"], inplace= True)
lga_mapping_df.head()



,LGA_CODE_2020,LGA_NAME_2020,AREA_ALBERS_SQKM
0,29499,No usual address (Vic.),0.0000
1,20110,Alpine (S),0.0127
2,20110,Alpine (S),0.0127
3,20110,Alpine (S),0.0105
4,20110,Alpine (S),0.0105


In [50]:
lga_mapping_df.describe()

,LGA_CODE_2020,AREA_ALBERS_SQKM
count,85014.000000,85014.000000
mean,23904.760357,2.675978
std,2117.225178,27.492022
min,20110.000000,0.000000
25%,22170.000000,0.021900
50%,23670.000000,0.034800
75%,25430.000000,0.068400
max,29799.000000,5172.342600


In [51]:
lga_mapping_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85014 entries, 0 to 85013
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   LGA_CODE_2020     85014 non-null  int64  
 1   LGA_NAME_2020     85014 non-null  object 
 2   AREA_ALBERS_SQKM  85014 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.9+ MB


In [52]:
#Importing The Victorian DHHS COVID data

dhhs_covid_df = pd.read_csv("https://www.dhhs.vic.gov.au/ncov-covid-cases-by-lga-source-csv")
dhhs_covid_df.head()

,diagnosis_date,Postcode,acquired,Localgovernmentarea
0,2020-03-11,3217,Travel overseas,Greater Geelong (C)
1,2020-03-08,3942,Travel overseas,Mornington Peninsula (S)
2,2020-03-09,3206,Travel overseas,Port Phillip (C)
3,2020-08-01,3754,Contact with a confirmed case,Whittlesea (C)
4,2020-03-01,3802,Travel overseas,Casey (C)


In [53]:
#add in  'year' column
dhhs_covid_df["year"]=dhhs_covid_df["diagnosis_date"].map(lambda x:x[0:4])

In [54]:
dhhs_covid_df.head()

,diagnosis_date,Postcode,acquired,Localgovernmentarea,year
0,2020-03-11,3217,Travel overseas,Greater Geelong (C),2020
1,2020-03-08,3942,Travel overseas,Mornington Peninsula (S),2020
2,2020-03-09,3206,Travel overseas,Port Phillip (C),2020
3,2020-08-01,3754,Contact with a confirmed case,Whittlesea (C),2020
4,2020-03-01,3802,Travel overseas,Casey (C),2020


In [55]:
dhhs_covid_df["month"]=dhhs_covid_df["diagnosis_date"].map(lambda x:x[5:7])

In [56]:
dhhs_covid_df.head()

,diagnosis_date,Postcode,acquired,Localgovernmentarea,year,month
0,2020-03-11,3217,Travel overseas,Greater Geelong (C),2020,03
1,2020-03-08,3942,Travel overseas,Mornington Peninsula (S),2020,03
2,2020-03-09,3206,Travel overseas,Port Phillip (C),2020,03
3,2020-08-01,3754,Contact with a confirmed case,Whittlesea (C),2020,08
4,2020-03-01,3802,Travel overseas,Casey (C),2020,03


In [57]:
dhhs_covid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1103993 entries, 0 to 1103992
Data columns (total 6 columns):
 #   Column               Non-Null Count    Dtype 
---  ------               --------------    ----- 
 0   diagnosis_date       1103993 non-null  object
 1   Postcode             1103993 non-null  int64 
 2   acquired             1103993 non-null  object
 3   Localgovernmentarea  1103993 non-null  object
 4   year                 1103993 non-null  object
 5   month                1103993 non-null  object
dtypes: int64(1), object(5)
memory usage: 50.5+ MB


In [58]:
dhhs_covid_df.describe()


,Postcode
count,1.103993e+06
mean,3.273654e+03
std,3.035823e+02
min,3.000000e+03
25%,3.060000e+03
50%,3.150000e+03
75%,3.350000e+03
max,3.999000e+03


In [59]:
#Display everything 
merged_df.set_option('display.max_colwidth', None)

NameError: name 'merged_df' is not defined

In [ ]:
#Merge df1 with df2 to get new data - Merge on LGA_CODE_2021

population_LGA

lga_mapping_df = pd.DataFrame(data1)
dhhs_covid_df = pd.DataFrame(data2)

newdf = df1.merge(df2, how='right')


In [ ]:
# Count total male / female - API 